In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from pandas import DataFrame as df
import pandas
import requests
from lxml import etree
import re
import time
import sys
import os
from bilibili_api import video

# 作者信息

In [116]:
mid=2140126087

url_1 = "https://api.bilibili.com/x/web-interface/card?mid=%d"%(mid)
url_1

'https://api.bilibili.com/x/web-interface/card?mid=2140126087'

In [117]:
author = {}
import json
res_1 = requests.get(url_1)
info_1 = json.loads(res_1.text)
#姓名、性别、视频数、粉丝数、总点赞数
author['name'] = info_1['data']['card']['name']
author['sex'] = info_1['data']['card']['sex']
video_num = info_1["data"]["archive_count"]
author['video_num'] = video_num
author['fans'] = info_1["data"]["follower"]
author['like_num'] = info_1["data"]["like_num"]

In [118]:
author

{'name': '爱飞的野妞',
 'sex': '女',
 'video_num': 40,
 'fans': 34537,
 'like_num': 110339}

In [119]:
#总浏览量
url_2 = "https://api.bilibili.com/x/space/upstat?mid=%d"%(mid)
headers={
     'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:56.0) Gecko/20100101 Firefox/56.0',
     'Host': 'api.bilibili.com',
     'Cookie': 'SESSDATA=e17f7302%2C1688455619%2Cfa3a5*11'
   }
res_2 = requests.get(url_2,headers=headers)
info_2 = json.loads(res_2.text)
author['view_num'] = info_2["data"]["archive"]["view"]

In [120]:
#入驻时间
import math
import time
import datetime
video_num=author['video_num']
pn = math.ceil(int(video_num)/30)
url_3 = "https://api.bilibili.com/x/space/arc/search?mid=%d&pn=%d&ps=30&index=1"%(mid, pn)
res_3 = requests.get(url_3,headers=headers)
info_3 = json.loads(res_3.text)
timestamp = info_3['data']['list']['vlist'][-1]['created']
author["create_timestamp"] = timestamp
author["create_time"] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(timestamp))

In [121]:
#发布频率
timenow = time.strftime("%Y-%m-%d %H:%M:%S")
timepast = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(timestamp))
time1=datetime.datetime.strptime(timenow,"%Y-%m-%d %H:%M:%S")
time2 = datetime.datetime.strptime(timepast, "%Y-%m-%d %H:%M:%S")
create_totalDays = (time1-time2).days
author['create_totalDays'] = create_totalDays
author['publish_frequency(days)'] = ('%.2f' % (create_totalDays/video_num))

In [122]:
author

{'name': '爱飞的野妞',
 'sex': '女',
 'video_num': 40,
 'fans': 34537,
 'like_num': 110339,
 'view_num': 3348583,
 'create_timestamp': 1653472106,
 'create_time': '2022-05-25 17:48:26',
 'create_totalDays': 343,
 'publish_frequency(days)': '8.57'}

# 作者主页视频

In [8]:
#从视频链接中提取BV号
def getBvid(url):
    bvid_pattern = re.compile(r"BV[0-9a-zA-Z]{10}")
    bvid = bvid_pattern.search(url).group(0)
    return bvid

In [9]:
url = "https://space.bilibili.com/%d/video"%(mid)
browser = webdriver.Chrome()
browser.get(url)
source = browser.page_source.encode("utf-8")
soup = BeautifulSoup(source,'html.parser')
items = soup.find_all(class_='small-item fakeDanmu-item')

In [10]:
bvs=[]
for item in items:
    infos= item.find_all(class_="title")
    for info in infos:
        url_video =info.get('href')
        bv=getBvid(url_video)
        bvs.append(bv)
bvs

['BV14T41117L8',
 'BV1nA411o7Zt',
 'BV1SG4y1A7Bd',
 'BV1uW4y137tZ',
 'BV1UA411Z7ZM',
 'BV11G4y1w7xa',
 'BV1wP411F7nG',
 'BV1Cg41147fA',
 'BV1WV4y1A7JL',
 'BV1k84y1y7Ek',
 'BV1B84y1v7SH',
 'BV1iG411c79s',
 'BV1SW4y1E7Yb',
 'BV1jT411K79V',
 'BV1sT411T7sG',
 'BV1XB4y1J7XW',
 'BV1TD4y1z7Jq',
 'BV1oe4y1o7sZ',
 'BV17V4y1s7od',
 'BV14a411P7Gb',
 'BV1JG4y1v7av',
 'BV1nG4y1i768',
 'BV1bf4y1o7RJ',
 'BV1Qr4y177SR',
 'BV1JS4y1p7H9',
 'BV1HB4y1D7aC',
 'BV1Nr4y1x7oq',
 'BV18Y4y1L7DW',
 'BV1dT4y1B7d6',
 'BV1au411r7X6']

# 基本视频信息

In [11]:
#获得视频信息的字典
def getInfo(bvid):
    #oid=getoid(bvid)
    video_info = video.get_video_info(bvid=bvid)
    #获取视频cid
    cid = video_info["cid"]
    #获取作者mid
    #mid=video_info['owner']['mid']
    #标题
    title = video_info["title"]
    #视频时长
    duration=video_info['duration']
    #播放量
    play = video_info["stat"]["view"]
    #弹幕数量
    danmu_num = video_info["stat"]["danmaku"]
    #评论量
    reply= video_info["stat"]["reply"]
    #收藏量
    favorite = video_info["stat"]["favorite"]
    #投币量
    coin= video_info["stat"]["coin"]
    #转发量
    share= video_info["stat"]["share"]
    #点赞量
    like= video_info["stat"]["like"]
    #作者
    author = video_info["owner"]["name"]
    info ={"作者":author,"BV":bvid,"ChatId":cid,"标题":title,"视频时长":duration,"播放量":play,"弹幕数量":danmu_num,"评论量":reply,"收藏量":favorite,"投币量":coin,"转发量":share,"点赞量":like}
    return info

In [12]:
data = df([])
for i in bvs:
    data = data.append(getInfo(i),ignore_index=True)

In [13]:
#视频信息
data

,作者,BV,ChatId,标题,视频时长,播放量,弹幕数量,评论量,收藏量,投币量,转发量,点赞量
0,papi酱,BV14T41117L8,973505963.0,过了虎年是兔年，金兔送喜闹新春！欢迎收看《姜家新春特别节目》！,231.0,983882.0,2151.0,1193.0,6339.0,21668.0,4867.0,145500.0
1,papi酱,BV1nA411o7Zt,964256576.0,2022年度未播出视频大放送——如何把小商品拍成买不起的样子 （无意义的慢镜头+随便说点啥=...,111.0,1311162.0,2596.0,1948.0,6162.0,21821.0,5694.0,162989.0
2,papi酱,BV1SG4y1A7Bd,963997319.0,2022年度未播出视频大放送——假如相亲对象的好友是HR（2,197.0,1540133.0,2498.0,1845.0,6352.0,18346.0,9571.0,186530.0
3,papi酱,BV1uW4y137tZ,962478723.0,2022年度未播出视频大放送——有时候我真的不懂这个世界,88.0,1979015.0,1598.0,2177.0,9128.0,16560.0,5382.0,252825.0
4,papi酱,BV1UA411Z7ZM,960964352.0,有些粉丝爱明星 像极了妈妈爱孩子,185.0,1824453.0,2969.0,3324.0,10052.0,23987.0,11189.0,266006.0
5,papi酱,BV11G4y1w7xa,959587937.0,一年一度的办公室礼物交换大会,1178.0,1470354.0,9534.0,2373.0,10026.0,24098.0,4954.0,200908.0
6,papi酱,BV1wP411F7nG,950719770.0,喜欢只是一时的，双标却是一世的。,177.0,5163656.0,9538.0,4721.0,20756.0,22125.0,14426.0,452284.0
7,papi酱,BV1Cg41147fA,941337137.0,这年头，追客户就跟追对象一样（1）,230.0,1878437.0,2022.0,1320.0,7058.0,21013.0,7396.0,215652.0
8,papi酱,BV1WV4y1A7JL,925842726.0,当我坐不同交通工具时 我在想什么,157.0,3891287.0,2405.0,2798.0,11480.0,14855.0,3773.0,377683.0
9,papi酱,BV1k84y1y7Ek,893456341.0,职场人的内心独白番外篇之——到底什么意思啊？总监大人！,154.0,2339809.0,2171.0,2704.0,9939.0,20601.0,27002.0,230380.0


In [22]:
path = "./data/"
#data.to_csv(path+"bilbilizuozhe.csv",encoding="utf_8_sig",index=False,header=True)

In [35]:
import pandas as pd
import numpy as np
data1= pd.read_csv(r"C:\Users\CANDY\代码\PycharmProjects\data\bilbilizuozhe.csv",sep=',')

# 判别

In [14]:
data1=data

In [15]:
data1.insert(1,'广告置顶',0)
data1.insert(2,'广告赞助商',0)
data1.insert(3,'广告弹幕',0)

In [16]:
data1

,作者,广告置顶,广告赞助商,广告弹幕,BV,ChatId,标题,视频时长,播放量,弹幕数量,评论量,收藏量,投币量,转发量,点赞量
0,papi酱,0,0,0,BV14T41117L8,973505963.0,过了虎年是兔年，金兔送喜闹新春！欢迎收看《姜家新春特别节目》！,231.0,983882.0,2151.0,1193.0,6339.0,21668.0,4867.0,145500.0
1,papi酱,0,0,0,BV1nA411o7Zt,964256576.0,2022年度未播出视频大放送——如何把小商品拍成买不起的样子 （无意义的慢镜头+随便说点啥=...,111.0,1311162.0,2596.0,1948.0,6162.0,21821.0,5694.0,162989.0
2,papi酱,0,0,0,BV1SG4y1A7Bd,963997319.0,2022年度未播出视频大放送——假如相亲对象的好友是HR（2,197.0,1540133.0,2498.0,1845.0,6352.0,18346.0,9571.0,186530.0
3,papi酱,0,0,0,BV1uW4y137tZ,962478723.0,2022年度未播出视频大放送——有时候我真的不懂这个世界,88.0,1979015.0,1598.0,2177.0,9128.0,16560.0,5382.0,252825.0
4,papi酱,0,0,0,BV1UA411Z7ZM,960964352.0,有些粉丝爱明星 像极了妈妈爱孩子,185.0,1824453.0,2969.0,3324.0,10052.0,23987.0,11189.0,266006.0
5,papi酱,0,0,0,BV11G4y1w7xa,959587937.0,一年一度的办公室礼物交换大会,1178.0,1470354.0,9534.0,2373.0,10026.0,24098.0,4954.0,200908.0
6,papi酱,0,0,0,BV1wP411F7nG,950719770.0,喜欢只是一时的，双标却是一世的。,177.0,5163656.0,9538.0,4721.0,20756.0,22125.0,14426.0,452284.0
7,papi酱,0,0,0,BV1Cg41147fA,941337137.0,这年头，追客户就跟追对象一样（1）,230.0,1878437.0,2022.0,1320.0,7058.0,21013.0,7396.0,215652.0
8,papi酱,0,0,0,BV1WV4y1A7JL,925842726.0,当我坐不同交通工具时 我在想什么,157.0,3891287.0,2405.0,2798.0,11480.0,14855.0,3773.0,377683.0
9,papi酱,0,0,0,BV1k84y1y7Ek,893456341.0,职场人的内心独白番外篇之——到底什么意思啊？总监大人！,154.0,2339809.0,2171.0,2704.0,9939.0,20601.0,27002.0,230380.0


In [17]:
#得到oid
def getoid(bv):
    resp=requests.get("https://www.bilibili.com/video/"+bv)
    obj=re.compile(f'"aid":(?P<id>.*?),"bvid":"{bv}"')     #在网页源代码里可以找到id，用正则获取到
    oid=obj.search(resp.text).group('id')
    #print('oid是'+oid)    #在程序运行时告诉我们已经获取到了参数oid
    return oid

In [18]:
#判别关键词
def is_contain(A):
    special=['羊毛','优惠','元','福利','https:/','链接']
    literal_string =A
    for i in special:
        if i in literal_string:
            return True

In [19]:
#置顶评论
def gettopcomment(oid):
    try:                      #需要判断是否有置顶的评论，函数返回值为主评论计数
                                  #如果有，之后的评论从2开始计数，则返回2
                                  #如果没有，之后的评论从1开始计数，则返回1
        param={
                        "jsonp": "jsonp",
                        "type":1,
                        "oid":oid,   
                        "next":0
                            }
        url="https://api.bilibili.com/x/v2/reply/main"    
        resp=requests.get(url,params=param)
        data=json.loads(resp.text)['data']
        content=data['top']['upper']['content']['message']
        #print(content)
        return content
    except:
        return 0      

In [20]:
#广告置顶
for i in range(len(data1)):
    bv=data1['BV'][i]
    oid=getoid(bv)
    content=gettopcomment(oid)
    if content!=0:
        if is_contain(content):
            data1["广告置顶"][i]=1

In [21]:
#广告赞助
for i in range(len(data1)):
    bv=data1['BV'][i]
    video_info = video.get_video_info(bvid=bv)
    try:
        print(video_info["staff"][1]['title'])
        if video_info["staff"][1]['title']=='赞助商':
            data1["广告赞助商"][i]=1
    except:
        pass

In [22]:
#广告弹幕
import requests
from lxml import etree
def get_danmu(BVID,cid):
    danmus = video.get_danmaku(bvid=BVID)
    url = 'https://comment.bilibili.com/%d.xml'%(cid)
    # 发送请求
    response = requests.get(url)
    xml = etree.fromstring(response.content)
    # 解析数据
    dms= xml.xpath("/i/d/text()")
    return dms
for i in range(len(data1)):
    bv=data1['BV'][i]
    cid=data1['ChatId'][i]
    danmus=get_danmu(bv,cid)
    for j in danmus:
        if ('猝不及防' in j )or('快跑'in j )or( '图穷匕见'in j)or ('我就知道'in j)or ('味道不对'in j):
            data1['广告弹幕'][i]=1
            break

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [23]:
data1

,作者,广告置顶,广告赞助商,广告弹幕,BV,ChatId,标题,视频时长,播放量,弹幕数量,评论量,收藏量,投币量,转发量,点赞量
0,papi酱,0,0,1,BV14T41117L8,973505963.0,过了虎年是兔年，金兔送喜闹新春！欢迎收看《姜家新春特别节目》！,231.0,983882.0,2151.0,1193.0,6339.0,21668.0,4867.0,145500.0
1,papi酱,0,0,1,BV1nA411o7Zt,964256576.0,2022年度未播出视频大放送——如何把小商品拍成买不起的样子 （无意义的慢镜头+随便说点啥=...,111.0,1311162.0,2596.0,1948.0,6162.0,21821.0,5694.0,162989.0
2,papi酱,0,0,0,BV1SG4y1A7Bd,963997319.0,2022年度未播出视频大放送——假如相亲对象的好友是HR（2,197.0,1540133.0,2498.0,1845.0,6352.0,18346.0,9571.0,186530.0
3,papi酱,0,0,0,BV1uW4y137tZ,962478723.0,2022年度未播出视频大放送——有时候我真的不懂这个世界,88.0,1979015.0,1598.0,2177.0,9128.0,16560.0,5382.0,252825.0
4,papi酱,0,0,0,BV1UA411Z7ZM,960964352.0,有些粉丝爱明星 像极了妈妈爱孩子,185.0,1824453.0,2969.0,3324.0,10052.0,23987.0,11189.0,266006.0
5,papi酱,0,0,0,BV11G4y1w7xa,959587937.0,一年一度的办公室礼物交换大会,1178.0,1470354.0,9534.0,2373.0,10026.0,24098.0,4954.0,200908.0
6,papi酱,0,0,1,BV1wP411F7nG,950719770.0,喜欢只是一时的，双标却是一世的。,177.0,5163656.0,9538.0,4721.0,20756.0,22125.0,14426.0,452284.0
7,papi酱,0,0,1,BV1Cg41147fA,941337137.0,这年头，追客户就跟追对象一样（1）,230.0,1878437.0,2022.0,1320.0,7058.0,21013.0,7396.0,215652.0
8,papi酱,0,0,0,BV1WV4y1A7JL,925842726.0,当我坐不同交通工具时 我在想什么,157.0,3891287.0,2405.0,2798.0,11480.0,14855.0,3773.0,377683.0
9,papi酱,0,0,1,BV1k84y1y7Ek,893456341.0,职场人的内心独白番外篇之——到底什么意思啊？总监大人！,154.0,2339809.0,2171.0,2704.0,9939.0,20601.0,27002.0,230380.0


In [24]:
df=data1[(data1["广告置顶"]==1)|(data1["广告赞助商"]==1)|(data1["广告弹幕"]==1)]

In [25]:
df=df.reset_index(drop=True)

In [26]:
df

,作者,广告置顶,广告赞助商,广告弹幕,BV,ChatId,标题,视频时长,播放量,弹幕数量,评论量,收藏量,投币量,转发量,点赞量
0,papi酱,0,0,1,BV14T41117L8,973505963.0,过了虎年是兔年，金兔送喜闹新春！欢迎收看《姜家新春特别节目》！,231.0,983882.0,2151.0,1193.0,6339.0,21668.0,4867.0,145500.0
1,papi酱,0,0,1,BV1nA411o7Zt,964256576.0,2022年度未播出视频大放送——如何把小商品拍成买不起的样子 （无意义的慢镜头+随便说点啥=...,111.0,1311162.0,2596.0,1948.0,6162.0,21821.0,5694.0,162989.0
2,papi酱,0,0,1,BV1wP411F7nG,950719770.0,喜欢只是一时的，双标却是一世的。,177.0,5163656.0,9538.0,4721.0,20756.0,22125.0,14426.0,452284.0
3,papi酱,0,0,1,BV1Cg41147fA,941337137.0,这年头，追客户就跟追对象一样（1）,230.0,1878437.0,2022.0,1320.0,7058.0,21013.0,7396.0,215652.0
4,papi酱,0,0,1,BV1k84y1y7Ek,893456341.0,职场人的内心独白番外篇之——到底什么意思啊？总监大人！,154.0,2339809.0,2171.0,2704.0,9939.0,20601.0,27002.0,230380.0
5,papi酱,0,0,1,BV1B84y1v7SH,886055216.0,啊，这就是中年的前奏吗？,169.0,3592369.0,7176.0,5102.0,11471.0,18230.0,17318.0,319925.0
6,papi酱,0,0,1,BV1SW4y1E7Yb,871628319.0,假如相亲对象的好友是HR,214.0,4246913.0,5572.0,3892.0,30011.0,72188.0,89467.0,401762.0
7,papi酱,0,0,1,BV1sT411T7sG,839363817.0,本期视频灵感来源@papi酱,272.0,3112387.0,8661.0,5767.0,26317.0,87543.0,9074.0,444412.0
8,papi酱,0,0,1,BV1XB4y1J7XW,830982437.0,去年拍了没发的视频，这两天翻出来一看，嚯！跟新的一样！,165.0,2995412.0,3195.0,2518.0,11440.0,33857.0,13200.0,308542.0
9,papi酱,0,0,1,BV1TD4y1z7Jq,826385276.0,老年大学手机课2,214.0,1948710.0,3140.0,1861.0,7738.0,21421.0,11369.0,212297.0


In [27]:
path = "./data/"
df.to_csv(path+"识别papidemo.csv",encoding="utf_8_sig",index=False,header=True)

二次识别

In [ ]:
import glob
def get_real_time(v_str):
    total_seconds= int(v_str.split('.')[0])
    return total_seconds
import pandas as pd
import numpy as np
data1= pd.read_csv(r"C:\Users\CANDY\代码\PycharmProjects\data\情感值papidemo.csv",sep=',')
data1

In [ ]:
from bs4 import BeautifulSoup as BS
import time
import pandas as pd
import numpy as np
for i in range(len(data1)):
    cid=int(data1['ChatId'][i])
    cpath=r"C:\Users\CANDY\Desktop\毕设\弹幕\papi\%d.xml"%(cid)
    files = sorted(glob.glob(cpath))
    for index, filename in enumerate(files, 1):
        file = open(filename, encoding='utf-8').read()
        soup = BS(file, 'xml')
    danmu_list=soup.find_all('d')
    data =pd.DataFrame([])
    for d in danmu_list:
        time=get_real_time(d['p'])
        content=d.text
        info={"时间s":time,"弹幕内容":content}
        data = data.append(info,ignore_index=True)
    count=0
    for j in range(len(data)):            
        if ('猝不及防' in data["弹幕内容"][j])or('快跑' in data["弹幕内容"][j])or ('图穷匕见' in data["弹幕内容"][j])or('我就知道' in data["弹幕内容"][j])or('味道不对' in data["弹幕内容"][j]):
            count=count+1
    if count<5:
        data1['广告弹幕'][i]=0

In [ ]:
df=data1[(data1["广告置顶"]==1)|(data1["广告赞助商"]==1)|(data1["广告弹幕"]==1)]
df=df.reset_index(drop=True)
df

In [ ]:
path = "./data/"
df.to_csv(path+"二次筛选.csv",encoding="utf_8_sig",index=False,header=True)

# 评论情感值

In [29]:
import pandas as pd
import numpy as np
data1= pd.read_csv(r"C:\Users\CANDY\代码\PycharmProjects\data\识别papidemo.csv",sep=',')
data1.insert(6,'评论情感值',0.0)

In [30]:
from collections import defaultdict
import os
import re
import jieba
import codecs

In [31]:
#jieba分词后去除停用词
def seg_word(sentence):
    seg_list = jieba.cut(sentence)
    seg_result = []
    for i in seg_list:
        seg_result.append(i)
    stopwords = set()
    with open('stopwords.txt','r',encoding='utf-8') as fr:
        for i in fr:
            stopwords.add(i.strip())
    return list(filter(lambda x :x not in stopwords,seg_result))
#找出文本中的情感词、否定词和程度副词
def classify_words(word_list):
    #读取情感词典文件
    sen_file = open(r'C:\Users\CANDY\代码\BosonNLP_sentiment_score.txt','r+',encoding='utf-8')
    #获取词典文件内容
    sen_list = sen_file.readlines()
    #创建情感字典
    sen_dict = defaultdict()
    #读取词典每一行的内容，将其转换成字典对象，key为情感词，value为其对应的权重
    for i in sen_list:
        if len(i.split(' '))==2:
            sen_dict[i.split(' ')[0]] = i.split(' ')[1]
 
    #读取否定词文件
    not_word_file = open(r'C:\Users\CANDY\代码\否定词.txt','r+',encoding='utf-8')
    not_word_list = not_word_file.readlines()
    #读取程度副词文件
    degree_file = open(r'C:\Users\CANDY\代码\程度副词.txt','r+',encoding='utf-8')
    degree_list = degree_file.readlines()
    degree_dict = defaultdict()
    for i in degree_list:
        degree_dict[i.split(',')[0]] = i.split(',')[1]
    sen_word = dict()
    not_word = dict()
    degree_word = dict()
    #分类
    for i in range(len(word_list)):
        word = word_list[i]
        if word in sen_dict.keys() and word not in not_word_list and word not in degree_dict.keys():
            # 找出分词结果中在情感字典中的词
            sen_word[i] = sen_dict[word]
        elif word in not_word_list and word not in degree_dict.keys():
            # 分词结果中在否定词列表中的词
            not_word[i] = -1
        elif word in degree_dict.keys():
            # 分词结果中在程度副词中的词
            degree_word[i]  = degree_dict[word]
 
 
    #关闭打开的文件
    sen_file.close()
    not_word_file.close()
    degree_file.close()
    #返回分类结果
    return sen_word,not_word,degree_word
#计算情感词的分数
def score_sentiment(sen_word,not_word,degree_word,seg_result):
    #权重初始化为1
    W = 1
    score = 0
    #情感词下标初始化
    sentiment_index = -1
    #情感词的位置下标集合
    sentiment_index_list = list(sen_word.keys())
    #遍历分词结果
    for i in range(0,len(seg_result)):
        #如果是情感词
        if i in sen_word.keys():
            #权重*情感词得分
            score += W*float(sen_word[i])
            #情感词下标加一，获取下一个情感词的位置
            sentiment_index += 1
            if sentiment_index < len(sentiment_index_list)-1:
                #判断当前的情感词与下一个情感词之间是否有程度副词或否定词
                for j in range(sentiment_index_list[sentiment_index],sentiment_index_list[sentiment_index+1]):
                    #更新权重，如果有否定词，权重取反
                    if j in not_word.keys():
                        W *= -1
                    elif j in degree_word.keys():
                        W *= float(degree_word[j])
        #定位到下一个情感词
        if sentiment_index < len(sentiment_index_list)-1:
            i = sentiment_index_list[sentiment_index+1]
    return score
 
#计算得分
def sentiment_score(sentence):
    #1.对文档分词
    seg_list = seg_word(sentence)
    #2.将分词结果转换成字典，找出情感词、否定词和程度副词
    sen_word,not_word,degree_word = classify_words(seg_list)
    #3.计算得分
    score = score_sentiment(sen_word,not_word,degree_word,seg_list)
    return score

In [32]:
def b2a(bid):
    """把哔哩哔哩视频的bv号转成av号"""
    table = 'fZodR9XQDSUm21yCkr6zBqiveYah8bt4xsWpHnJE7jL5VG3guMTKNPAwcF'
    tr = {}
    for i in range(58):
        tr[table[i]] = i
    s = [11, 10, 3, 8, 4, 6]
    r = 0
    for i in range(6):
        r += tr[bid[s[i]]] * 58 ** i
    aid = (r - 8728348608) ^ 177451812
    return aid

In [33]:
def response_f(bv_id, next=0, mode=3):
    api_url = 'https://api.bilibili.com/x/v2/reply/main'
    url = 'https://www.bilibili.com/video/' + bv_id
    av = b2a(bv_id) # 先转av号
    headers = {
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
        # 需定期更换cookie，否则location爬不到
        'cookie': cookie,
        'origin': 'https://www.bilibili.com',
        'referer': 'https://www.bilibili.com/video/BV1FG4y1Z7po/?spm_id_from=333.337.search-card.all.click&vd_source=69a50ad969074af9e79ad13b34b1a548',
        'sec-ch-ua': '"Chromium";v="106", "Microsoft Edge";v="106", "Not;A=Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.47'
  }
    # 定义所拆解内容
    data = {
        'jsonp': 'jsonp',
        'next': next,  # 页码
        'type': '1',
        'oid': av,  # av号
        'mode': mode,  # 1:楼层大前小后, 2:时间晚前早后, 3:热门评论
        'plat': '1',
        '_': str(time.time() * 1000)[:13],  # 时间戳
    }
    response = requests.get(api_url, headers=headers, params=data)
    # 中文，不定义编码格式大概率会乱码
    response.encoding = 'utf-8'
 
    # 将得到的json文本转化为可读json，这段是复制的
    if 'code' in response.text:
        c_json = json.loads(response.text)
    else:
        c_json = {'code': -1}
    if c_json['code'] != 0:
        print('json error!')
        print(response.status_code)
        print(response.text)
        return 0  # 读取错误
    return c_json
def response_r(bv, rpid, pn=1):
    r_api_url = 'https://api.bilibili.com/x/v2/reply/reply'
    url = 'https://www.bilibili.com/video/' + bv
    av = b2a(bv)
    headers = {
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
        # 需定期更换cookie，否则location爬不到
        'cookie': cookie,
        'origin': 'https://www.bilibili.com',
        'referer': 'https://www.bilibili.com/video/BV1FG4y1Z7po/?spm_id_from=333.337.search-card.all.click&vd_source=69a50ad969074af9e79ad13b34b1a548',
        'sec-ch-ua': '"Chromium";v="106", "Microsoft Edge";v="106", "Not;A=Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.47'
     }
    data = {
        'jsonp': 'jsonp',
        'pn': pn,  # page number
        'type': '1',
        'oid': av,
        'ps': '10',
        'root': rpid,  # 父评论的rpid
        '_': str(time.time() * 1000)[:13],  # 时间戳
    }
    response = requests.get(r_api_url, headers=headers, params=data)
    response.encoding = 'utf-8'
 
    # 加载得到的json
    if 'code' in response.text:
        r_cjson = json.loads(response.text)
    else:
        r_cjson = {'code': -1}
    if r_cjson['code'] != 0:
        print('error!')
        print(response.status_code)
        print(response.text)
        return 0  # 读取错误
    return r_cjson

In [34]:
def parse_comment_r(bv, rpid, df):
    """
    解析子评论json
    :param bv: bv号
    :param rpid: 父评论的id
    :param df: pandas datagram，作为本程序的数据传递方式
    :return: 返回修改后的df
    """
 
    cr_json = response_r(bv, rpid)['data']
    count = cr_json['page']['count']
    for pn in range(1, count // 10 + 2):
        time.sleep(random.uniform(0, 1))
        print('p%d %d  ' % (pn, count), end='\r')
        cr_json = response_r(bv, rpid, pn=pn)['data']
        cr_list = cr_json['replies']
        if cr_list:  # 有时'replies'为'None'
            for i in range(len(cr_list)):
                try:
                    location = cr_list[i]['reply_control']['location']
                except:
                    location = ""
                comment_temp = [{
                    'like': cr_list[i]['like'],  # 赞数
                    'mid': cr_list[i]['member']['mid'],  # uid
                    'content': '"' + cr_list[i]['content']['message'] + '"',  # 子评论
                }]  # 保留需要的内容
                df2 = pd.DataFrame(comment_temp)
                df = pd.concat([df, df2], axis=0, ignore_index=True)
    return df
def parse_comment_f(bv, df):
    c_json = response_f(bv, mode=comment_mode)
    if c_json:
        # 父评论总数
        try:
            count_all = c_json['data']['cursor']['all_count']
            print('comments:%d' % count_all)
        except KeyError:
            print('KeyError, 该视频可能没有评论!')
            return '0', '2'  # 找不到键值
    else:
        print('json错误')
        return '1', '0'  # json错误
 
    # 开始序号
    count_next = 0
 
    # 存放原始json
    all_json = ''
    for page in range(6):
        time.sleep(random.uniform(0, 1))
        print('page:%d' % (page + 1))
 
        c_json = response_f(bv, count_next, mode=comment_mode)
        all_json += str(c_json) + '\n'
        if not c_json:
            return 1  # json错误
        count_next = c_json['data']['cursor']['next']  # 下一个的序号
 
        # 评论列表
        c_list = c_json['data']['replies']
 
        # 有评论,就进入下面的循环保存
        if c_list:
            for i in range(len(c_list)):
                try:
                    location = c_list[i]['reply_control']['location']
                except:
                    location = ""
                comment_temp = [{
                    'like': c_list[i]['like'],  # 赞数
                    'mid': c_list[i]['member']['mid'],  # uid
                    'content': c_list[i]['content']['message'],  # 评论内容
                }]  # 保留需要的内容
                # 若有子评论,记录rpid,爬取子评论
                replies = False
                replies = False
                if c_list[i]['rcount'] or ('replies' in c_list[i] and c_list[i]['replies']):
                    replies = True
                    rpid = c_list[i]['rpid']
                    df = parse_comment_r(bv, rpid, df) # 如果有回复评论,爬取子评论
 
                df2 = pd.DataFrame(comment_temp)
                df = pd.concat([df, df2], axis=0, ignore_index=True)
 
            if c_json['data']['cursor']['is_end']:
                print('读取完毕,结束')
                # 为最后一个json,结束爬取
                break
        else:
            print('评论为空,结束!')
            break
        time.sleep(0.2)
    return df, all_json

In [35]:
def main(bv):
    data0 = [{ 'like': '','mid':'', 'content': ''}]  # 行首
    df = pd.DataFrame(data0)
    df, all_json = parse_comment_f(bv, df)
    df = df.drop(index=0) # 这种方式会有空行，把它干掉
    return df

In [37]:
"""def commentscore(df):
    #df.drop(df.loc[df['mid']=='323024456'].index, inplace=True)
    #df.reset_index(drop=True)
    scores=[]
    for i in range(len(df)):
        comment=df['content'][i+1]
        like=df['like'][i+1]+1#多少人支持
        score=sentiment_score(comment)
        postscore=score*like
        scores.append(postscore)
    mean=sum(scores)/len(scores)
    return mean"""
def commentscore(df):
    mid=1532165
    scores=[]
    for i in range(len(df)):
        if df['mid'][i+1]!=mid:
            comment=df['content'][i+1]
            like=df['like'][i+1]+1#多少人支持
            score=sentiment_score(comment)
            postscore=score*like
            scores.append(postscore)
    mean=sum(scores)/len(scores)
    return mean

In [38]:
import random
if __name__ == "__main__":
    
    comment_mode = 3
    cookie='buvid3=5BE5558E-99ED-497D-A83F-B7CC54102920143102infoc; buvid_fp_plain=undefined; buvid4=43B5BD6A-A479-737D-4C41-E42DDE86FBF494344-022012917-LKyJvix6SQivideHLEgMVA%3D%3D; i-wanna-go-back=-1; CURRENT_BLACKGAP=0; blackside_state=0; nostalgia_conf=-1; CURRENT_QUALITY=0; CURRENT_FNVAL=4048; b_nut=100; _uuid=6DF7641F-1C63-176A-85D3-398A5657CA41075811infoc; bsource=search_baidu; rpdid=|(J|)l|YkR|J0J\'uY~Jull|uR; fingerprint=0051bf13b69428bb1adfc27fbe69293b; SESSDATA=248295ee%2C1688707060%2Cd7e91%2A12; bili_jct=fa3193c10e50277f82b2d54e05320421; DedeUserID=491214274; DedeUserID__ckMd5=ed18ee4d6f7b3c2e; sid=6r6c26e9; b_ut=5; is-2022-channel=1; innersign=1; b_lsid=A8ED910FB_18590626C3C; buvid_fp=0051bf13b69428bb1adfc27fbe69293b; PVID=6'
    for i in range(5,len(data1)):
        bv=data1['BV'][i]
        df=main(bv)
        mean=commentscore(df)
        data1['评论情感值'][i]=mean
        print('== over! ==')

comments:5102
page:1
page:2   
page:3 
page:4 
page:5
page:6


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\CANDY\AppData\Local\Temp\jieba.cache
Loading model cost 0.883 seconds.
Prefix dict has been built successfully.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


== over! ==
comments:3892
page:1
page:2   
page:3 
page:4
page:5
page:6
== over! ==
comments:5767
page:1
page:22  
page:3 
page:4 
page:5 
page:6
== over! ==
comments:2518
page:1
page:2   
page:3 
page:4
page:5
page:6
== over! ==
comments:1861
page:1
page:2   
page:3 
page:4
page:5
page:6
== over! ==
comments:5304
page:1
page:2     
page:3 
page:4 
page:5
page:6 
== over! ==
comments:5827
page:1
page:2   
page:3 
page:4 
page:5
page:6 
== over! ==
comments:5299
page:1
page:2   
page:3   
page:4 
page:5 
page:6 
== over! ==
comments:3166
page:1
page:2   
page:3 
page:4 
page:5
page:6
== over! ==


In [40]:
data1

,作者,广告置顶,广告赞助商,广告弹幕,BV,ChatId,评论情感值,标题,视频时长,播放量,弹幕数量,评论量,收藏量,投币量,转发量,点赞量
0,papi酱,0,0,1,BV14T41117L8,973505963.0,0,过了虎年是兔年，金兔送喜闹新春！欢迎收看《姜家新春特别节目》！,231.0,983882.0,2151.0,1193.0,6339.0,21668.0,4867.0,145500.0
1,papi酱,0,0,1,BV1nA411o7Zt,964256576.0,0,2022年度未播出视频大放送——如何把小商品拍成买不起的样子 （无意义的慢镜头+随便说点啥=...,111.0,1311162.0,2596.0,1948.0,6162.0,21821.0,5694.0,162989.0
2,papi酱,0,0,1,BV1wP411F7nG,950719770.0,0,喜欢只是一时的，双标却是一世的。,177.0,5163656.0,9538.0,4721.0,20756.0,22125.0,14426.0,452284.0
3,papi酱,0,0,1,BV1Cg41147fA,941337137.0,0,这年头，追客户就跟追对象一样（1）,230.0,1878437.0,2022.0,1320.0,7058.0,21013.0,7396.0,215652.0
4,papi酱,0,0,1,BV1k84y1y7Ek,893456341.0,0,职场人的内心独白番外篇之——到底什么意思啊？总监大人！,154.0,2339809.0,2171.0,2704.0,9939.0,20601.0,27002.0,230380.0
5,papi酱,0,0,1,BV1B84y1v7SH,886055216.0,69,啊，这就是中年的前奏吗？,169.0,3592369.0,7176.0,5102.0,11471.0,18230.0,17318.0,319925.0
6,papi酱,0,0,1,BV1SW4y1E7Yb,871628319.0,325,假如相亲对象的好友是HR,214.0,4246913.0,5572.0,3892.0,30011.0,72188.0,89467.0,401762.0
7,papi酱,0,0,1,BV1sT411T7sG,839363817.0,448,本期视频灵感来源@papi酱,272.0,3112387.0,8661.0,5767.0,26317.0,87543.0,9074.0,444412.0
8,papi酱,0,0,1,BV1XB4y1J7XW,830982437.0,829,去年拍了没发的视频，这两天翻出来一看，嚯！跟新的一样！,165.0,2995412.0,3195.0,2518.0,11440.0,33857.0,13200.0,308542.0
9,papi酱,0,0,1,BV1TD4y1z7Jq,826385276.0,-35,老年大学手机课2,214.0,1948710.0,3140.0,1861.0,7738.0,21421.0,11369.0,212297.0


In [41]:
path = "./data/"
data1.to_csv(path+"情感值papidemo.csv",encoding="utf_8_sig",index=False,header=True)

# 弹幕及弹幕情感值